In [1]:
import torch
from torchvision import models
from collections import OrderedDict
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from sklearn.metrics import precision_score,f1_score
import cupy as cp
from __future__ import print_function
import argparse
import random # to set the python random seed
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from collections import OrderedDict
import pandas as pd
import os
import shutil
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms
#from albumentations import Compose, ShiftScaleRotate, Resize, CenterCrop
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import warnings
# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
wandb.login()

wandb: Currently logged in as: alqnatri-ahmed (use `wandb login --relogin` to force relogin)


True

In [ ]:
def choose_mid_slices(LABEL_DIR,EXPORT_DIR):
    lables = pd.read_csv(LABEL_DIR)
    start, end = 0,0
    data = {'Name':[],
           'Target':[]} 
    for i, row in tqdm(lables.iterrows()):
        if start == 0:
            start = i
        if row["StudyInstance"] == lables["StudyInstance"][start]:
            #print(f'index {i} file_name {row["filename"]} pataint_name {row["PatientID"]}')
            continue
        else:
            end = i-1
            mid = int((end+start)/2)
            
            data["Name"].append(lables["filename"][mid+2])
            data["Target"].append(lables["any"][mid+2])          
            
            data["Name"].append(lables["filename"][mid])
            data["Target"].append(lables["any"][mid])
            
            data["Name"].append(lables["filename"][mid+1])
            data["Target"].append(lables["any"][mid+1])
        
            data["Name"].append(lables["filename"][mid+3])
            data["Target"].append(lables["any"][mid+3])
            
            data["Name"].append(lables["filename"][mid+4])
            data["Target"].append(lables["any"][mid+4])
            
            start = i
    
    new_df = pd.DataFrame(data)
    new_df.to_csv(EXPORT_DIR,index=False)
    
    
#EXPORT_DIR = 'C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/CSV/test_mid_slice.csv'    
#choose_mid_slices('C:/Users/alqna/Documents/Bachelor/Bachelor_Data/csv/train_meta_id_seriser.csv',EXPORT_DIR)

In [ ]:
def move_files():
    LABEL_DIR = 'C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/CSV/test_subtype_classes_1.csv'
    files = pd.read_csv(LABEL_DIR)
    src='C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/TEST/'
    dst='C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/TEST_MULTI/'

    for i, row in tqdm(files.iterrows()):
        shutil.copy(src+row['Name'], dst+row['Name'])
#move_files()

In [ ]:
def create_targets(DATA_DIR, TARGETS_FILE, EXPORT_DIR):
    p = [f for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]
    df = pd.read_csv(TARGETS_FILE)

    data = {'Name':[], 
            'Target':[]} 
    counts = [0,0,0,0,0]
    flags = [0,0,0,0,0] 
    for i in tqdm(p):
        index = df.index[df['filename'] == i[:-4]]
        any_result = df["any"].iloc[index].to_numpy()
        if any_result[0] == 0:
            continue
            #data["Target"].append(any_result[0])
            #data["Name"].append(i)
        else:
            for column in df:
                if column == 'any' or column == 'filename':
                    continue
                else:
                    result = df[column].iloc[index].to_numpy()
                    if result[0] == 0:
                        continue
                    else:
                        index_count = df.columns.get_loc(column) - 2
                        flags[index_count] = 1
            min_index = -1
            for flag in range(len(flags)):
                if flags[flag] == 0:
                    continue
                else:
                    if min_index == -1 :
                        min_index = flag
                    else:
                        if counts[flag] < counts[min_index]:
                            min_index = flag
            counts[min_index] = counts[min_index] + 1
                    
            data["Target"].append(min_index+1)
            data["Name"].append(i)
            flags = [0,0,0,0,0]
            
    new_df = pd.DataFrame(data)
    new_df.to_csv(EXPORT_DIR,index=False)

    
#DATA_DIR='C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/TRAIN/'
#TARGETS_FILE='C:/Users/alqna/Documents/Bachelor/Bachelor_Data/csv/standard.csv'
#EXPORT_DIR='C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/CSV/train_subtype_classes.csv'
#create_targets(DATA_DIR, TARGETS_FILE, EXPORT_DIR)

In [ ]:
def create_multi_lable(Wanted, TARGETS_FILE, EXPORT_DIR):
    #p = [f for f in os.listdir(DATA_DIR) if os.path.isfile(os.path.join(DATA_DIR, f))]
    df = pd.read_csv(TARGETS_FILE)
    df2 = pd.read_csv(Wanted)
    df['filename'] = df['filename'].astype(str) + '.dcm'
    new_df = df[~df['filename'].isin(df2['Name'])]
    new_df_2 = df[~df['filename'].isin(new_df['filename'])]
    new_df_2.to_csv(EXPORT_DIR,index=False)
    #return new_df_2


Wanted = 'C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/CSV/train_subtype_classes.csv'
TARGETS_FILE = 'C:/Users/alqna/Documents/Bachelor/Bachelor_Data/csv/standard.csv'
EXPORT_DIR ='C:/Users/alqna/Documents/Bachelor/rsna-intracranial-hemorrhage-detection/Big_Data/CSV/train_multilabel.csv'
#a = create_multi_lable(Wanted,TARGETS_FILE,EXPORT_DIR)


In [3]:
def sigmoid_window(dcm, window_center, window_width, U=1.0, eps=(1.0 / 255.0)):
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img = cp.array(np.array(img))
    ue = cp.log((U / eps) - 1.0)
    W = (2 / window_width) * ue
    b = ((-2 * window_center) / window_width) * ue
    z = W * img + b
    img = U / (1 + cp.power(np.e, -1.0 * z))
    img = (img - cp.min(img)) / (cp.max(img) - cp.min(img))
    return cp.asnumpy(img)

def sigmoid_bsb_window(img):
    brain_img = sigmoid_window(img, 40, 80)
    subdural_img = sigmoid_window(img, 80, 200)
    bone_img = sigmoid_window(img, 600, 2000)
    
    bsb_img = np.zeros((3,brain_img.shape[0], brain_img.shape[1]))
    bsb_img[0, :, :] = brain_img
    bsb_img[1, :, :] = subdural_img
    bsb_img[2, :, :] = bone_img
    bsb_img = torch.tensor(bsb_img, dtype=torch.float32)
    return bsb_img


    
def load_dicom(path):
    img = pydicom.dcmread(path)
    return img


def preprocess_data(inputs):
    # normalization
    #inputs = np.resize(inputs,(512,512))
    """
    inputs = np.expand_dims(inputs, axis=0)
    inputs = np.repeat(inputs,3,axis=0)
    inputs -= inputs.mean()
    inputs[inputs!=0] /= np.std(inputs[inputs!=0])
    """

    #inputs = bsb_window(inputs)
    inputs = sigmoid_bsb_window(inputs)
    return inputs

In [4]:
class DicomDataset(Dataset):
    def __init__(self ,rootPath, labelsPath, transform=None):
        # load only all data paths
        self.rootPath = rootPath
        self.transform = transform
        self.labels = pd.read_csv(labelsPath, index_col=0)
        p = [f for f in os.listdir(rootPath) if os.path.isfile(os.path.join(rootPath, f))]
        all_paths = np.array(p)
        self.paths = all_paths
        
    def __len__(self):
        # define size of dataset for dataloader
        return len(self.paths)

    def __getitem__(self, item):
        # load data from path[item]
        inputs = load_dicom(os.path.join(self.rootPath, self.paths[item-1]))
        
        outputs = {
                'epidural':torch.tensor(self.labels.loc[self.paths[item-1]]["epidural"], dtype=torch.float32),
                'intraparenchymal':torch.tensor(self.labels.loc[self.paths[item-1]]["intraparenchymal"], dtype=torch.float32),
                'intraventricular':torch.tensor(self.labels.loc[self.paths[item-1]]["intraventricular"], dtype=torch.float32),
                'subarachnoid':torch.tensor(self.labels.loc[self.paths[item-1]]["subarachnoid"], dtype=torch.float32),
                'subdural':torch.tensor(self.labels.loc[self.paths[item-1]]["subdural"], dtype=torch.float32)
        }

        inputs = preprocess_data(inputs)
        if self.transform:
            inputs = self.transform(inputs)
        # return tuple
        return inputs, outputs

In [ ]:
class BC_classification_model_resnet152(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet152(pretrained=True)
        modules = list(self.model.children())[:-1]      
        self.model = nn.Sequential(*modules)
        for parameter in self.model.parameters():
            parameter.requires_grad = False
        
        self.classifier_BC = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
   
    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, start_dim=1)
        return {
            'BC':  self.classifier_BC(x),

        }
    
    def get_loss(self, net_output, ground_truth):
        loss = F.binary_cross_entropy_with_logits(net_output['BC'], ground_truth['BC'].unsqueeze(1).to(device))
      
        return loss, {
            'BC': Epidural_loss,
        }
    
    def predict(self, images):
        outputs = self(images)
        _, predicted = torch.max(outputs, 1)
        return predicted

In [5]:
class Multi_classification_model_resnet152(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet152(pretrained=True)
        modules = list(self.model.children())[:-1]      
        self.model = nn.Sequential(*modules)
        for parameter in self.model.parameters():
            parameter.requires_grad = False
        
        self.classifier_Epidural = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
        self.classifier_Intraparenchymal = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
        self.classifier_Intraventricular = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
        self.classifier_Subarachnoid = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
        self.classifier_Subdural = nn.Sequential(
                          nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.3),
                          nn.Linear(512, 1),
                          #nn.LogSoftmax(dim=1)
                          )
    def forward(self, x):
        x = self.model(x)
        x = torch.flatten(x, start_dim=1)
        return {
            'epidural':  self.classifier_Epidural(x),
            'intraparenchymal': self.classifier_Intraparenchymal(x),
            'intraventricular': self.classifier_Intraventricular(x),
            'subarachnoid': self.classifier_Subarachnoid(x),
            'subdural': self.classifier_Subdural(x)
        }
    
    def get_loss(self, net_output, ground_truth):
        Epidural_loss = F.binary_cross_entropy_with_logits(net_output['epidural'], ground_truth['epidural'].unsqueeze(1).to(device))
        Intraparenchymal_loss = F.binary_cross_entropy_with_logits(net_output['intraparenchymal'], ground_truth['intraparenchymal'].unsqueeze(1).to(device))
        Intraventricular_loss = F.binary_cross_entropy_with_logits(net_output['intraventricular'], ground_truth['intraventricular'].unsqueeze(1).to(device))
        Subarachnoid_loss = F.binary_cross_entropy_with_logits(net_output['subarachnoid'], ground_truth['subarachnoid'].unsqueeze(1).to(device))
        Subdural_loss = F.binary_cross_entropy_with_logits(net_output['subdural'], ground_truth['subdural'].unsqueeze(1).to(device))
        loss = Epidural_loss + Intraparenchymal_loss + Intraventricular_loss + Subarachnoid_loss + Subdural_loss
        return loss/5, {
            'epidural': Epidural_loss,
            'intraparenchymal': Intraparenchymal_loss,
            'intraventricular': Intraventricular_loss,
            'subarachnoid': Subarachnoid_loss,
            'subdural': Subdural_loss
        }
    
    def predict(self, images):
        outputs = self(images)
        _, predicted = torch.max(outputs, 1)
        return predicted

In [ ]:
# https://github.com/ufoym/imbalanced-dataset-sampler/
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                

        self.indices = list(range(len(dataset))) if indices is None else indices

        self.callback_get_label = callback_get_label

        self.num_samples = len(self.indices)  if num_samples is None else num_samples
            
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            label_to_count[label] = label_to_count.get(label, 0) + 1
            """
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
            """  
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        
        return dataset[idx][1].item()
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))
        #return iter(torch.multinomial(self.weights, self.num_samples, replacement=True).tolist())
    def __len__(self):
        return self.num_samples

In [ ]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True):
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [6]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    y_pred_tag = torch.ravel(y_pred_tag)
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [7]:
def train_func(args, model, device, train_loader, optimizer):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        
        optimizer.zero_grad()
        output = model.forward(data)
        loss_train, losses_train = model.get_loss(output, target)
        total_loss += loss_train.item() * data.size(0)
        loss_train.backward()
        optimizer.step() 
              
    total_loss /= len(train_loader.dataset)
    wandb.log({ # "Test Accuracy": 100. * correct / (len(test_loader)),
    "Train Loss": total_loss})

In [8]:
def test_func(args, model, device, test_loader, classes):
    model.eval()

    total_loss = 0
    example_images = []
    total_acc = 0
    acc_dict = {}
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            output = model.forward(data)
            loss_test, losses_test = model.get_loss(output, target)
            total_loss += loss_test.item() * data.size(0)
            
            acc = 0
            pred_list=''
            truth_list=''
            for label in classes:
                acc_dict[label] = binary_acc(output[label].to("cpu"), target[label])
                acc += acc_dict[label]
                
                pred = torch.ravel(torch.round(torch.sigmoid(output[label])))
                if int(pred[0].item()) == 1:
                    #pred_list.append(label)
                    pred_list+=label+'/'
                    
                if int(target[label][0].item()) == 1:
                    #truth_list.append(label)
                    truth_list+=label+'/'
                #print("Pred: {} Truth: {}".format(pred, target[label]))
            #print("Pred: {} Truth: {}".format(pred_list, truth_list))
            example_images.append(wandb.Image(data[0],
                                caption="Pred: {} Truth: {}".format(pred_list, truth_list)))

            acc/=5
            total_acc += acc

        del data, target, output
        torch.cuda.empty_cache()
        #print("acc ", total_acc/len(test_loader))
        wandb.log({
            "Example Image":example_images,
            "Test Loss": total_loss/len(test_loader.dataset),
            "Test Accuracy": total_acc/len(test_loader)}) 
        return total_loss
        

In [9]:
def save_checkpoint(model, epoch, optimizer, loss, PATH):
    checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'model': Multi_classification_model_resnet152()
    }            
    torch.save(checkpoint, PATH)

In [ ]:
def load_checkpoint(filepath): 
    
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

#model = load_checkpoint('checkpoint.pth')
#print(model)


In [10]:
wandb.init(project="multi-lable-classification")
wandb.watch_called = False 

In [13]:
config = wandb.config          # Initialize config
config.batch_size = 32         # input batch size for training (default: 64)
config.test_batch_size = 32    # input batch size for testing (default: 1000)
config.epochs = 50             # number of epochs to train (default: 10)
config.lr = 0.01               # learning rate (default: 0.01)
config.momentum = 0.9          # SGD momentum (default: 0.5) 
config.no_cuda = False         # disables CUDA training
config.seed = 42               # random seed (default: 42)
config.log_interval = 1     # how many batches to wait before logging training status

In [14]:
def main():
    use_cuda = not config.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    print("device", device)
    torch.cuda.empty_cache()
    kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
    
    torch.manual_seed(config.seed) 
    torch.backends.cudnn.deterministic = True
    
 
    
    DATA_DIR_TRAIN = '../Final Bachelor/Data/MultiLabel_Data/TRAIN'
    DATA_DIR_TEST = '../Final Bachelor/Data/MultiLabel_Data/TRAIN'
    DATA_TARGET_TRAIN = '../Final Bachelor/Data/MultiLabel_Data/TRAIN/train_multilabel.csv'
    DATA_TARGET_TEST = '../Final Bachelor/Data/MultiLabel_Data/TRAIN/test_multilabel.csv'
    
    data_transforms = transforms.Compose([
        transforms.Resize(265),
        transforms.CenterCrop(224)
    ])
    
    train = DicomDataset(rootPath=DATA_DIR_TRAIN, labelsPath=DATA_TARGET_TRAIN, transform=data_transforms)
    test  = DicomDataset(rootPath=DATA_DIR_TEST, labelsPath=DATA_TARGET_TEST, transform=data_transforms)
    
    # data loader
    train_loader = DataLoader(dataset=train, batch_size=config.batch_size, drop_last=True, num_workers = 0, pin_memory=True, shuffle=True)
    test_loader = DataLoader(dataset=test, batch_size=config.test_batch_size, drop_last=True,  num_workers = 0, pin_memory=True, shuffle=True)

    model = Multi_classification_model_resnet152().to(device)
    criterion = nn.NLLLoss()
    
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)
    #optimizer = optim.Adam(model.fc.parameters(), lr=config.lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',patience=2)
    wandb.watch(model, log="all")
    
    PATH = '../Final Bachelor/Model/multi_label.pth'
    classes = ('epidural','intraparenchymal','intraventricular','subarachnoid','subdural')
    
    for epoch in tqdm(range(1, config.epochs + 1)):
        train_func(config, model, device, train_loader, optimizer)
        loss_val = test_func(config, model, device, test_loader,classes)
        scheduler.step(loss_val)
        if epoch % 5 == 0:
            save_checkpoint(model, epoch, optimizer, loss_val, PATH)
    
    wandb.save(os.path.join(wandb.run.dir, "multi_label.pth"))
    
    BC_model = load_checkpoint('../Final Bachelor/Model/binary.h5')
    Multi_model = load_checkpoint('../Final Bachelor/Model/multi_label.pth')
    image = '../Final Bachelor/Data/Binary_Data/TEST/ID_000c8cc79.dcm'
    Binary_predicted = BC_model.predict(image)
    if Binary_predicted == 1:
        print("ICH Positive")
        Multi_predicted = Multi_model.predict(image)
        ans = []
        for label in classes:
            if Multi_predicted[label] == 1:
                ans.append(label)
            print(ans)
            
if __name__ == '__main__':
    main()

device cuda:0


  0%|          | 0/50 [00:00<?, ?it/s]

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
